# Librerias y Configuraciones

In [1]:
import glob
from bs4 import BeautifulSoup
import os
import time
import datetime
from selenium import webdriver# I need to web scrap in chrome
from selenium.webdriver.support.select import Select # I need to select menu options
from random import randint
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)

# Funciones

In [2]:
def TransNum(Texto):
    Salida=Texto.replace(",","")
    return Salida
def leerarchivo(lugar):
    archi=open(lugar)
    temp='we'
    cuadros=""
    while temp != '':#lee todo
        temp=archi.readline()
        cuadros=cuadros+temp
    archi.close()
    return cuadros
def leerColumna(archivo,Sufijo):
    b=leerarchivo(archivo)
    soup=BeautifulSoup(b)
    Columns = []
    temp=[]
    table = soup.find_all('table')
    table_body = table[2].find('thead')
    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        Columns= ['Age', 'Month', 'Site']+[ele for ele in cols if ele]
        for k in range(7,len(Columns)):
            Columns[k]=Columns[k]+Sufijo    
    return(Columns)
def leerData(archivo,Columns,patron):
        temp2=archivo.replace("./Data/","").replace(".html","").split("_")
        b=leerarchivo(archivo)
        soup=BeautifulSoup(b)
        data = []
        temp=[]
        table = soup.find_all('table')
        #print(table[2])  
        table_body = table[2].find('tbody')
        rows = table_body.find_all('tr')
        for row in rows:
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            temp=[ele for ele in cols if ele]
            
            if 'ND' in set(temp[4:]) and len(set(temp[4:]))>1:
                data.append(temp2[:3]+temp)
        df = pd.DataFrame.from_records(data, columns=Columns)
        for ii in Columns[7:]:
            df[ii]=df[ii].apply(TransNum)
            df[ii]=pd.to_numeric(df[ii], errors='coerce')
        #df.info()
        df=df.drop(columns=['Variable Calculada', 'Total Residencial%s'%(patron),'Total No Residencial%s'%(patron)])
        return(df)
def SetPruebas(Data):
    Data.info()
    Data["Empresa"].value_counts()
def color_negative_red(val):
    if val < 0:
        color = 'red' 
    else:
        color ='black'
    
    return 'color: %s' % color 

def color_significant(val):
    if abs(val) > 0.3:
        color = 'yellow' 
    else:
        color = 'white' 
    return  'background-color: %s' % color 
def initSelenium():
    sistemaop = os.name
    if sistemaop=="posix":
        driver = webdriver.Chrome('./chromedriver')  # Optional argument, if not specified will search path.
    else: 
        driver = webdriver.Chrome('./chromedriver.exe')  # Optional argument, if not specified will search path.


# Descarga de datos

In [4]:
url="http://reportes.sui.gov.co/fabricaReportes/frameSet.jsp?idreporte=ele_com_096"
#initSelenium()
sistemaop = os.name
if sistemaop=="posix":
    driver = webdriver.Chrome('./chromedriver')  # Optional argument, if not specified will search path.
else: 
    driver = webdriver.Chrome('./chromedriver.exe')  # Optional argument, if not specified will search path.

for i in range(0,17):#Counter Age
    time_delay = randint(10,19)
    time.sleep(time_delay)
    driver.get(url)#Goto web
    dheader=driver.find_element_by_xpath("//*[@id='header']")#Search Head
    driver.switch_to.frame(dheader)#Manipulate Head
    inputElementi = driver.find_element_by_xpath("/html/body/form/table/tbody/tr[4]/td/table/tbody/tr[1]/td[2]/select/option[%s]"%(str(i+1)))
    age=inputElementi.text
    for ii in range(3):
        inputElementii = driver.find_element_by_xpath("/html/body/form/table/tbody/tr[4]/td/table/tbody/tr[3]/td[2]/select/option[%s]"%(str(ii+1)))
        site=inputElementii.text
        for iii in [1,2,3]:
            inputElementiii = driver.find_element_by_xpath("/html/body/form/table/tbody/tr[4]/td/table/tbody/tr[7]/td[2]/select/option[%s]"%(str(iii)))
            data=inputElementiii.text
            for iiii in range(12):
                try:
                    inputElementiiii = driver.find_element_by_xpath("/html/body/form/table/tbody/tr[4]/td/table/tbody/tr[2]/td[2]/select/option[%s]"%(str(iiii+1)))
                    month=inputElementiiii.text
                    TEMP=age+"_"+month+"_"+site+"_"+data
                except:
                    continue
                try:
                    if os.path.exists("./ele_com_096/Data/"+TEMP+".html"):
                        print("Existe ./ele_com_096/Data/"+TEMP+".html")
                        time.sleep(1)
                        continue
                    nomonths=["Anual",'Trimestre 1','Trimestre 2','Trimestre 3','Trimestre 4','Semestre 1','Semestre 2']
                    if month in nomonths:
                        continue  
                    driver.get(url)
                    dheader=driver.find_element_by_xpath("//*[@id='header']")
                    driver.switch_to.frame(dheader)
                    inputElementi = driver.find_element_by_xpath("/html/body/form/table/tbody/tr[4]/td/table/tbody/tr[1]/td[2]/select/option[%s]"%(str(i+1)))
                    inputElementi.click()
                    inputElementii = driver.find_element_by_xpath("/html/body/form/table/tbody/tr[4]/td/table/tbody/tr[3]/td[2]/select/option[%s]"%(str(ii+1)))
                    inputElementii.click()
                    inputElementiii = driver.find_element_by_xpath("/html/body/form/table/tbody/tr[4]/td/table/tbody/tr[7]/td[2]/select/option[%s]"%(str(iii)))
                    inputElementiii.click()
                    inputElementiiii = driver.find_element_by_xpath("/html/body/form/table/tbody/tr[4]/td/table/tbody/tr[2]/td[2]/select/option[%s]"%(str(iiii+1)))
                    inputElementiiii.click()
                    inputElement = driver.find_element_by_xpath("/html/body/form/table/tbody/tr[3]/td[2]/table/tbody/tr/td/input[2]")
                    time.sleep(time_delay+5)
                    inputElement.click()
                    time.sleep(time_delay)
                    driver.switch_to.default_content()
                    dreport=driver.find_element_by_xpath("//*[@id='report']")
                    driver.switch_to.frame(dreport)
                    html = driver.page_source
                    file2 = open("./ele_com_096/Data/"+TEMP+".html","w+")
                    file2.write(html)
                    file2.close()
                    driver.switch_to.default_content()
                    dheader=driver.find_element_by_xpath("//*[@id='header']")
                    driver.switch_to.frame(dheader)
                except:
                    print(TEMP)

Existe ./ele_com_096/Data/2003_Enero_Rural_Suscriptores.html
Existe ./ele_com_096/Data/2003_Febrero_Rural_Suscriptores.html
Existe ./ele_com_096/Data/2003_Marzo_Rural_Suscriptores.html
Existe ./ele_com_096/Data/2003_Abril_Rural_Suscriptores.html
Existe ./ele_com_096/Data/2003_Mayo_Rural_Suscriptores.html
Existe ./ele_com_096/Data/2003_Junio_Rural_Suscriptores.html
Existe ./ele_com_096/Data/2003_Julio_Rural_Suscriptores.html
Existe ./ele_com_096/Data/2003_Agosto_Rural_Suscriptores.html
Existe ./ele_com_096/Data/2003_Septiembre_Rural_Suscriptores.html
Existe ./ele_com_096/Data/2003_Octubre_Rural_Suscriptores.html
Existe ./ele_com_096/Data/2003_Noviembre_Rural_Suscriptores.html
Existe ./ele_com_096/Data/2003_Diciembre_Rural_Suscriptores.html
Existe ./ele_com_096/Data/2003_Enero_Rural_Consumo.html
Existe ./ele_com_096/Data/2003_Febrero_Rural_Consumo.html
Existe ./ele_com_096/Data/2003_Marzo_Rural_Consumo.html
Existe ./ele_com_096/Data/2003_Abril_Rural_Consumo.html
Existe ./ele_com_096/Data

Existe ./ele_com_096/Data/2004_Octubre_Rural_Consumo.html
Existe ./ele_com_096/Data/2004_Noviembre_Rural_Consumo.html
Existe ./ele_com_096/Data/2004_Diciembre_Rural_Consumo.html
Existe ./ele_com_096/Data/2004_Enero_Rural_Valor Consumo.html
Existe ./ele_com_096/Data/2004_Febrero_Rural_Valor Consumo.html
Existe ./ele_com_096/Data/2004_Marzo_Rural_Valor Consumo.html
Existe ./ele_com_096/Data/2004_Abril_Rural_Valor Consumo.html
Existe ./ele_com_096/Data/2004_Mayo_Rural_Valor Consumo.html
Existe ./ele_com_096/Data/2004_Junio_Rural_Valor Consumo.html
Existe ./ele_com_096/Data/2004_Julio_Rural_Valor Consumo.html
Existe ./ele_com_096/Data/2004_Agosto_Rural_Valor Consumo.html
Existe ./ele_com_096/Data/2004_Septiembre_Rural_Valor Consumo.html
Existe ./ele_com_096/Data/2004_Octubre_Rural_Valor Consumo.html
Existe ./ele_com_096/Data/2004_Noviembre_Rural_Valor Consumo.html
Existe ./ele_com_096/Data/2004_Diciembre_Rural_Valor Consumo.html
Existe ./ele_com_096/Data/2004_Enero_Urbano_Suscriptores.html

Existe ./ele_com_096/Data/2005_Junio_Urbano_Suscriptores.html
Existe ./ele_com_096/Data/2005_Julio_Urbano_Suscriptores.html
Existe ./ele_com_096/Data/2005_Agosto_Urbano_Suscriptores.html
Existe ./ele_com_096/Data/2005_Septiembre_Urbano_Suscriptores.html
Existe ./ele_com_096/Data/2005_Octubre_Urbano_Suscriptores.html
Existe ./ele_com_096/Data/2005_Noviembre_Urbano_Suscriptores.html
Existe ./ele_com_096/Data/2005_Diciembre_Urbano_Suscriptores.html
Existe ./ele_com_096/Data/2005_Enero_Urbano_Consumo.html
Existe ./ele_com_096/Data/2005_Febrero_Urbano_Consumo.html
Existe ./ele_com_096/Data/2005_Marzo_Urbano_Consumo.html
Existe ./ele_com_096/Data/2005_Abril_Urbano_Consumo.html
Existe ./ele_com_096/Data/2005_Mayo_Urbano_Consumo.html
Existe ./ele_com_096/Data/2005_Junio_Urbano_Consumo.html
Existe ./ele_com_096/Data/2005_Julio_Urbano_Consumo.html
Existe ./ele_com_096/Data/2005_Agosto_Urbano_Consumo.html
Existe ./ele_com_096/Data/2005_Septiembre_Urbano_Consumo.html
Existe ./ele_com_096/Data/2005

Existe ./ele_com_096/Data/2006_Marzo_Urbano_Valor Consumo.html
Existe ./ele_com_096/Data/2006_Abril_Urbano_Valor Consumo.html
Existe ./ele_com_096/Data/2006_Mayo_Urbano_Valor Consumo.html
Existe ./ele_com_096/Data/2006_Junio_Urbano_Valor Consumo.html
Existe ./ele_com_096/Data/2006_Julio_Urbano_Valor Consumo.html
Existe ./ele_com_096/Data/2006_Agosto_Urbano_Valor Consumo.html
Existe ./ele_com_096/Data/2006_Septiembre_Urbano_Valor Consumo.html
Existe ./ele_com_096/Data/2006_Octubre_Urbano_Valor Consumo.html
Existe ./ele_com_096/Data/2006_Noviembre_Urbano_Valor Consumo.html
Existe ./ele_com_096/Data/2006_Diciembre_Urbano_Valor Consumo.html
Existe ./ele_com_096/Data/2006_Enero_Centro Poblado_Suscriptores.html
Existe ./ele_com_096/Data/2006_Febrero_Centro Poblado_Suscriptores.html
Existe ./ele_com_096/Data/2006_Marzo_Centro Poblado_Suscriptores.html
Existe ./ele_com_096/Data/2006_Abril_Centro Poblado_Suscriptores.html
Existe ./ele_com_096/Data/2006_Mayo_Centro Poblado_Suscriptores.html
Exis

Existe ./ele_com_096/Data/2007_Octubre_Centro Poblado_Suscriptores.html
Existe ./ele_com_096/Data/2007_Noviembre_Centro Poblado_Suscriptores.html
Existe ./ele_com_096/Data/2007_Diciembre_Centro Poblado_Suscriptores.html
Existe ./ele_com_096/Data/2007_Enero_Centro Poblado_Consumo.html
Existe ./ele_com_096/Data/2007_Febrero_Centro Poblado_Consumo.html
Existe ./ele_com_096/Data/2007_Marzo_Centro Poblado_Consumo.html
Existe ./ele_com_096/Data/2007_Abril_Centro Poblado_Consumo.html
Existe ./ele_com_096/Data/2007_Mayo_Centro Poblado_Consumo.html
Existe ./ele_com_096/Data/2007_Junio_Centro Poblado_Consumo.html
Existe ./ele_com_096/Data/2007_Julio_Centro Poblado_Consumo.html
Existe ./ele_com_096/Data/2007_Agosto_Centro Poblado_Consumo.html
Existe ./ele_com_096/Data/2007_Septiembre_Centro Poblado_Consumo.html
Existe ./ele_com_096/Data/2007_Octubre_Centro Poblado_Consumo.html
Existe ./ele_com_096/Data/2007_Noviembre_Centro Poblado_Consumo.html
Existe ./ele_com_096/Data/2007_Diciembre_Centro Pobl

Existe ./ele_com_096/Data/2008_Mayo_Centro Poblado_Valor Consumo.html
Existe ./ele_com_096/Data/2008_Junio_Centro Poblado_Valor Consumo.html
Existe ./ele_com_096/Data/2008_Julio_Centro Poblado_Valor Consumo.html
Existe ./ele_com_096/Data/2008_Agosto_Centro Poblado_Valor Consumo.html
Existe ./ele_com_096/Data/2008_Septiembre_Centro Poblado_Valor Consumo.html
Existe ./ele_com_096/Data/2008_Octubre_Centro Poblado_Valor Consumo.html
Existe ./ele_com_096/Data/2008_Noviembre_Centro Poblado_Valor Consumo.html
Existe ./ele_com_096/Data/2008_Diciembre_Centro Poblado_Valor Consumo.html
Existe ./ele_com_096/Data/2009_Enero_Rural_Suscriptores.html
Existe ./ele_com_096/Data/2009_Febrero_Rural_Suscriptores.html
Existe ./ele_com_096/Data/2009_Marzo_Rural_Suscriptores.html
Existe ./ele_com_096/Data/2009_Abril_Rural_Suscriptores.html
Existe ./ele_com_096/Data/2009_Mayo_Rural_Suscriptores.html
Existe ./ele_com_096/Data/2009_Junio_Rural_Suscriptores.html
Existe ./ele_com_096/Data/2009_Julio_Rural_Suscrip

Existe ./ele_com_096/Data/2010_Diciembre_Rural_Suscriptores.html
Existe ./ele_com_096/Data/2010_Enero_Rural_Consumo.html
Existe ./ele_com_096/Data/2010_Febrero_Rural_Consumo.html
Existe ./ele_com_096/Data/2010_Marzo_Rural_Consumo.html
Existe ./ele_com_096/Data/2010_Abril_Rural_Consumo.html
Existe ./ele_com_096/Data/2010_Mayo_Rural_Consumo.html
Existe ./ele_com_096/Data/2010_Junio_Rural_Consumo.html
Existe ./ele_com_096/Data/2010_Julio_Rural_Consumo.html
Existe ./ele_com_096/Data/2010_Agosto_Rural_Consumo.html
Existe ./ele_com_096/Data/2010_Septiembre_Rural_Consumo.html
Existe ./ele_com_096/Data/2010_Octubre_Rural_Consumo.html
Existe ./ele_com_096/Data/2010_Noviembre_Rural_Consumo.html
Existe ./ele_com_096/Data/2010_Diciembre_Rural_Consumo.html
Existe ./ele_com_096/Data/2010_Enero_Rural_Valor Consumo.html
Existe ./ele_com_096/Data/2010_Febrero_Rural_Valor Consumo.html
Existe ./ele_com_096/Data/2010_Marzo_Rural_Valor Consumo.html
Existe ./ele_com_096/Data/2010_Abril_Rural_Valor Consumo.h

Existe ./ele_com_096/Data/2011_Septiembre_Rural_Valor Consumo.html
Existe ./ele_com_096/Data/2011_Octubre_Rural_Valor Consumo.html
Existe ./ele_com_096/Data/2011_Noviembre_Rural_Valor Consumo.html
Existe ./ele_com_096/Data/2011_Diciembre_Rural_Valor Consumo.html
Existe ./ele_com_096/Data/2011_Enero_Urbano_Suscriptores.html
Existe ./ele_com_096/Data/2011_Febrero_Urbano_Suscriptores.html
Existe ./ele_com_096/Data/2011_Marzo_Urbano_Suscriptores.html
Existe ./ele_com_096/Data/2011_Abril_Urbano_Suscriptores.html
Existe ./ele_com_096/Data/2011_Mayo_Urbano_Suscriptores.html
Existe ./ele_com_096/Data/2011_Junio_Urbano_Suscriptores.html
Existe ./ele_com_096/Data/2011_Julio_Urbano_Suscriptores.html
Existe ./ele_com_096/Data/2011_Agosto_Urbano_Suscriptores.html
Existe ./ele_com_096/Data/2011_Septiembre_Urbano_Suscriptores.html
Existe ./ele_com_096/Data/2011_Octubre_Urbano_Suscriptores.html
Existe ./ele_com_096/Data/2011_Noviembre_Urbano_Suscriptores.html
Existe ./ele_com_096/Data/2011_Diciembre_U

KeyboardInterrupt: 

# Stadistics DataBase

In [25]:
a=glob.glob("./Data/*.html")
len(a)
iii=[]
SetData=glob.glob("./Data/*_Suscriptores.html")#["./Data/2009_Enero_Rural_Suscriptores.html","./Data/2013_Enero_Rural_Suscriptores.html"]
cont3=len(SetData)
Total=cont3
for i in a:
    ii=i.replace("./Data/","").replace(".html","").split("_")
    iii=iii+ii
iii=set(iii)
for i in iii:
    a=len(glob.glob("./Data/*%s*.html"%(i)))
    print( "La etiqueta %s posee %s sets"%(i,a))
SetData=glob.glob("./Data/*_Suscriptores.html")
TypeData=["_Suscriptores","_Consumo","_Valor Consumo"]
for i1 in SetData: 
    cont3=cont3-1
    for i2 in TypeData:
        i3=i1.replace("_Suscriptores",i2)
        if not os.path.isfile(i3):
            print(i3)

La etiqueta Junio posee 153 sets
La etiqueta Julio posee 144 sets
La etiqueta Septiembre posee 144 sets
La etiqueta Diciembre posee 144 sets
La etiqueta 2006 posee 108 sets
La etiqueta 2015 posee 108 sets
La etiqueta 2019 posee 54 sets
La etiqueta 2004 posee 108 sets
La etiqueta 2018 posee 108 sets
La etiqueta Febrero posee 153 sets
La etiqueta Abril posee 153 sets
La etiqueta Octubre posee 144 sets
La etiqueta Consumo posee 1188 sets
La etiqueta 2009 posee 108 sets
La etiqueta 2003 posee 108 sets
La etiqueta Enero posee 153 sets
La etiqueta 2005 posee 108 sets
La etiqueta 2007 posee 108 sets
La etiqueta 2008 posee 108 sets
La etiqueta Centro Poblado posee 594 sets
La etiqueta 2016 posee 108 sets
La etiqueta 2017 posee 108 sets
La etiqueta 2014 posee 108 sets
La etiqueta Marzo posee 153 sets
La etiqueta Mayo posee 153 sets
La etiqueta Valor Consumo posee 594 sets
La etiqueta 2013 posee 108 sets
La etiqueta Urbano posee 594 sets
La etiqueta Suscriptores posee 594 sets
La etiqueta Rural 

# Carga de base de datos en Memoria

In [ ]:
SetData=glob.glob("./Data/*_Suscriptores.html")#["./Data/2009_Enero_Rural_Suscriptores.html","./Data/2013_Enero_Rural_Suscriptores.html"]
cont3=len(SetData)
Total=cont3
TiempoFin=0
TypeData=["_Suscriptores","_Consumo","_Valor Consumo"]
cont2=0
TiempoInit=datetime.datetime.now()
for i1 in SetData:
    
    cont3=cont3-1
    cont=0
    for i2 in TypeData:
        i3=i1.replace("_Suscriptores",i2)
        Temp1=leerColumna(i3,i2)
        Tabla=leerData(i3,Temp1,i2)
        if cont==0:
            FinalT=Tabla
            cont=cont+1
        else:
            FinalT=FinalT.merge(Tabla,on=['Age', 'Month', 'Site', 'Departamento', 'Municipio', 'Empresa'], suffixes=("ty", "fg"))
            cont=cont+1
    if cont2==0:
        FinalM=FinalT
        cont2=cont2+1

    else:
        FinalM=pd.concat([FinalM, FinalT])
        cont2=cont2+1
    TiempoFin=datetime.datetime.now()
    if cont3 %25==0:
        Estimador=(TiempoFin-TiempoInit)*cont3/(Total-cont3)
        TiemporPorData=(TiempoFin-TiempoInit)/(Total-cont3)
        print(Estimador,cont3, datetime.datetime.now()+ Estimador)

In [4]:
FinalM = pd.read_csv(
    "./DataSui.csv",      # relative python path to subdirectory
    sep=',',           # Tab-separated value file.
    #quotechar="'",        # single quote allowed as quote character
    #dtype={"salary": int},             # Parse the salary column as an integer 
    #usecols=['name', 'birth_date', 'salary'].   # Only load the three columns specified.
    #parse_dates=['birth_date'],     # Intepret the birth_date column as a date
    #skiprows=10,         # Skip the first 10 rows of the file
    na_values=''      # Take any '.' or '??' values as NA
)

# Pruebas en base de datos

In [5]:
FinalM.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 589476 entries, 0 to 589475
Data columns (total 37 columns):
Unnamed: 0                  589476 non-null int64
Age                         589476 non-null int64
Month                       589476 non-null object
Site                        589476 non-null object
Departamento                589476 non-null object
Municipio                   589476 non-null object
Empresa                     589476 non-null object
Estrato 1_Suscriptores      358401 non-null float64
Estrato 2_Suscriptores      352970 non-null float64
Estrato 3_Suscriptores      273596 non-null float64
Estrato 4_Suscriptores      135773 non-null float64
Estrato 5_Suscriptores      38453 non-null float64
Estrato 6_Suscriptores      29300 non-null float64
Industrial_Suscriptores     385700 non-null float64
Comercial_Suscriptores      452507 non-null float64
Oficial_Suscriptores        356473 non-null float64
Otros_Suscriptores          301487 non-null float64
Estrato 1_Consum

In [13]:
FinalM["Empresa"].value_counts()

ELECTRIFICADORA DEL CARIBE S.A. E.S.P.                                                        50881
EMPRESAS PÚBLICAS DE MEDELLIN E.S.P.                                                          48683
CODENSA S.A. ESP                                                                              39923
EMPRESA DE ENERGIA DE BOYACA S.A. ESP EMPRESA DE SERVICIOS PUBLICOS                           38162
ELECTRIFICADORA DE SANTANDER S.A. E.S.P.                                                      34469
VATIA S.A. E.S.P.                                                                             31992
EMGESA S.A. E.S.P.                                                                            24643
ENERTOTAL S.A. E.S.P.                                                                         24592
CENTRALES ELECTRICAS DE NARIÑO S.A. E.S.P.                                                    22237
EMPRESA DE ENERGIA DE CUNDINAMARCA S.A. ESP                                                   18492


In [14]:
FinalM["Departamento"].value_counts()

CUNDINAMARCA          92298
ANTIOQUIA             61662
BOYACA                47803
SANTANDER             43349
VALLE DEL CAUCA       35305
NARINO                27318
TOLIMA                25993
CAUCA                 25074
BOLIVAR               21909
CORDOBA               18236
NORTE DE SANTANDER    18007
MAGDALENA             17258
CESAR                 16548
ATLANTICO             16547
CALDAS                16184
HUILA                 14966
SUCRE                 14748
META                  13045
RISARALDA             11565
CASANARE               7929
CHOCO                  7573
LA GUAJIRA             7297
PUTUMAYO               7139
QUINDIO                6614
CAQUETA                6472
BOGOTA, D.C.           4512
ARAUCA                 3435
GUAVIARE                690
Name: Departamento, dtype: int64

In [15]:
FinalM["Municipio"].value_counts()

BOGOTA, D.C.              4512
CALI                      2949
MOSQUERA                  2853
BARRANQUILLA              2746
IBAGUE                    2716
MADRID                    2686
SOACHA                    2637
YUMBO                     2627
CARTAGO                   2621
BUCARAMANGA               2598
VILLAVICENCIO             2561
PEREIRA                   2515
TOCANCIPA                 2450
FUNZA                     2399
LA UNION                  2391
FACATATIVA                2377
CARTAGENA DE INDIAS       2376
MEDELLIN                  2271
PALMIRA                   2226
RIONEGRO                  2179
CALOTO                    2111
ZIPAQUIRA                 1994
PUERTO TEJADA             1991
PASTO                     1959
SANTANDER DE QUILICHAO    1903
BUENAVISTA                1901
SOGAMOSO                  1884
DOSQUEBRADAS              1850
VILLETA                   1836
ARMENIA                   1817
                          ... 
PUERTO CONCORDIA           232
CARTAGEN

In [16]:
FinalM["Site"].value_counts()

Urbano            346264
Rural             218182
Centro Poblado     25030
Name: Site, dtype: int64

In [17]:
FinalM["Month"].value_counts()

Marzo         51877
Junio         51658
Abril         50287
Febrero       49864
Mayo          49743
Enero         49698
Diciembre     49097
Septiembre    48983
Octubre       47322
Noviembre     47279
Agosto        47013
Julio         46655
Name: Month, dtype: int64

In [10]:
FinalM["Age"].value_counts()

2018    40048
2014    39574
2015    39210
2013    39104
2017    37686
2016    37549
2009    36912
2010    36557
2008    36055
2012    35867
2011    34288
2007    34013
2005    32604
2006    31259
2004    29875
2003    28422
2019    20453
Name: Age, dtype: int64

In [11]:
FinalM.describe()

,Unnamed: 0,Age,Estrato 1_Suscriptores,Estrato 2_Suscriptores,Estrato 3_Suscriptores,Estrato 4_Suscriptores,Estrato 5_Suscriptores,Estrato 6_Suscriptores,Industrial_Suscriptores,Comercial_Suscriptores,Oficial_Suscriptores,Otros_Suscriptores,Estrato 1_Consumo,Estrato 2_Consumo,Estrato 3_Consumo,Estrato 4_Consumo,Estrato 5_Consumo,Estrato 6_Consumo,Industrial_Consumo,Comercial_Consumo,Oficial_Consumo,Otros_Consumo,Estrato 1_Valor Consumo,Estrato 2_Valor Consumo,Estrato 3_Valor Consumo,Estrato 4_Valor Consumo,Estrato 5_Valor Consumo,Estrato 6_Valor Consumo,Industrial_Valor Consumo,Comercial_Valor Consumo,Oficial_Valor Consumo,Otros_Valor Consumo
count,589476.000000,589476.000000,358401.000000,352970.000000,273596.000000,135773.000000,38453.000000,29300.000000,385700.000000,452507.000000,356473.000000,301487.000000,3.584010e+05,3.529700e+05,2.735960e+05,1.357730e+05,3.845300e+04,2.930000e+04,3.857000e+05,4.525070e+05,3.564730e+05,3.014870e+05,3.584010e+05,3.529700e+05,2.735960e+05,1.357730e+05,3.845300e+04,2.930000e+04,3.857000e+05,4.525070e+05,3.564730e+05,3.014870e+05
mean,732.962826,2011.177174,955.535537,1005.174397,427.433329,116.082005,81.154058,40.529693,14.490106,117.094308,19.370457,11.804685,1.353993e+05,1.288846e+05,6.161207e+04,2.145632e+04,1.887986e+04,1.843798e+04,4.681593e+05,1.409138e+05,3.527172e+04,6.052657e+04,5.124485e+07,4.802749e+07,2.263265e+07,7.866181e+06,6.370348e+06,5.444713e+06,9.433657e+07,4.037410e+07,1.023302e+07,1.673194e+07
std,504.538326,4.698841,2565.715309,2670.927232,2606.302195,1004.773911,759.181814,461.052139,141.267594,862.191328,31.176022,32.990340,5.005851e+05,4.514595e+05,3.776750e+05,1.928860e+05,1.810602e+05,1.805777e+05,7.329771e+06,2.027498e+06,1.615522e+06,1.471674e+06,2.069299e+08,1.688535e+08,1.341778e+08,6.622839e+07,5.505058e+07,4.941752e+07,5.295898e+08,2.373676e+08,7.526500e+07,8.674258e+07
min,0.000000,2003.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,-1.630230e+05,-1.158920e+05,-4.813600e+04,-1.074000e+03,0.000000e+00,0.000000e+00,-1.145670e+08,-6.703726e+07,-1.249238e+08,-2.876387e+07,-1.056726e+09,-3.633487e+08,-4.216667e+08,-3.225949e+07,-1.886435e+06,-5.248107e+06,-4.658411e+08,-4.454326e+08,-6.058762e+08,-2.258890e+08
25%,308.000000,2007.000000,51.000000,154.000000,5.000000,2.000000,1.000000,1.000000,1.000000,3.000000,6.000000,1.000000,3.983000e+03,1.364300e+04,6.580000e+02,2.590000e+02,2.160000e+02,1.720000e+02,1.976000e+03,3.645000e+03,1.651000e+03,2.340000e+03,1.409454e+06,4.951490e+06,2.456814e+05,9.552000e+04,7.512100e+04,5.942200e+04,6.931515e+05,1.302666e+06,5.829390e+05,7.957155e+05
50%,681.000000,2011.000000,277.000000,460.000000,30.000000,5.000000,3.000000,2.000000,3.000000,15.000000,13.000000,4.000000,2.380800e+04,4.041800e+04,3.987000e+03,1.051000e+03,8.790000e+02,7.130000e+02,1.619400e+04,1.511100e+04,4.836000e+03,1.067200e+04,8.614426e+06,1.486310e+07,1.471238e+06,3.814180e+05,3.218420e+05,2.409580e+05,5.350856e+06,5.261728e+06,1.733144e+06,3.450950e+06
75%,1060.000000,2015.000000,903.000000,1062.000000,183.000000,23.000000,14.000000,7.000000,10.000000,68.000000,24.000000,11.000000,9.537900e+04,1.098440e+05,2.544225e+04,4.848000e+03,4.363000e+03,3.157500e+03,1.159658e+05,5.733950e+04,1.445900e+04,3.411450e+04,3.466096e+07,3.947058e+07,9.048966e+06,1.688411e+06,1.535365e+06,1.055632e+06,3.161764e+07,1.887409e+07,5.208616e+06,1.091377e+07
max,2211.000000,2019.000000,119463.000000,599175.000000,619346.000000,225132.000000,78768.000000,63078.000000,27178.000000,166945.000000,3940.000000,1717.000000,2.995478e+07,9.329723e+07,1.010801e+08,3.958043e+07,1.782942e+07,2.042590e+07,4.120560e+09,1.057159e+09,8.903453e+08,5.971390e+08,2.512088e+10,3.094194e+10,3.343275e+10,1.299253e+10,6.499592e+09,6.739433e+09,4.246230e+10,3.212789e+10,2.487364e+10,9.162810e+09


In [12]:
import matplotlib.pyplot as plt
FinalM.hist(bins=100, figsize=(90,90))
plt.savefig ('grafico01.png')

In [21]:
FinalM.corr(method="kendall" ).style.applymap(color_negative_red).applymap(color_significant)

,Unnamed: 0,Age,Estrato 1_Suscriptores,Estrato 2_Suscriptores,Estrato 3_Suscriptores,Estrato 4_Suscriptores,Estrato 5_Suscriptores,Estrato 6_Suscriptores,Industrial_Suscriptores,Comercial_Suscriptores,Oficial_Suscriptores,Otros_Suscriptores,Estrato 1_Consumo,Estrato 2_Consumo,Estrato 3_Consumo,Estrato 4_Consumo,Estrato 5_Consumo,Estrato 6_Consumo,Industrial_Consumo,Comercial_Consumo,Oficial_Consumo,Otros_Consumo,Estrato 1_Valor Consumo,Estrato 2_Valor Consumo,Estrato 3_Valor Consumo,Estrato 4_Valor Consumo,Estrato 5_Valor Consumo,Estrato 6_Valor Consumo,Industrial_Valor Consumo,Comercial_Valor Consumo,Oficial_Valor Consumo,Otros_Valor Consumo
Unnamed: 0,1,0.0893076,-0.0247456,0.0516741,0.164855,0.0900301,0.110469,0.0871937,-0.0469493,-0.0393589,0.0930942,-0.154783,0.0240713,0.0725334,0.170688,0.10388,0.201854,0.202417,0.152248,0.133856,0.149821,0.053723,0.0213706,0.0754596,0.174119,0.107187,0.205529,0.204596,0.155805,0.135415,0.154268,0.0577572
Age,0.0893076,1,0.0892768,0.0313622,-0.018465,-0.00819234,0.00715672,-0.0357081,0.0110199,0.0227505,0.035239,0.141597,0.0491942,0.013693,-0.0348272,-0.0584261,-0.0496405,-0.0917056,0.0399103,0.0851154,0.0362163,0.0806704,0.122283,0.114351,0.0264224,0.0123294,0.0188855,-0.0290092,0.0997912,0.165806,0.131596,0.160579
Estrato 1_Suscriptores,-0.0247456,0.0892768,1,0.168714,0.153036,0.111767,0.124708,0.0779485,0.087188,0.2629,0.245882,0.356348,0.797335,0.218884,0.187477,0.133522,0.1127,0.0818519,0.141122,0.26602,0.278127,0.244395,0.805345,0.232432,0.197566,0.145488,0.124522,0.0893106,0.147036,0.268973,0.279565,0.252885
Estrato 2_Suscriptores,0.0516741,0.0313622,0.168714,1,0.400372,0.22595,0.173218,-0.0254378,0.316981,0.453604,0.369953,0.237958,0.141851,0.768823,0.375064,0.196744,0.135315,-0.0229499,0.259967,0.409787,0.333877,0.237308,0.144663,0.764371,0.384264,0.204467,0.141923,-0.0206135,0.261808,0.4048,0.33072,0.24077
Estrato 3_Suscriptores,0.164855,-0.018465,0.153036,0.400372,1,0.386719,0.293841,0.0871157,0.283277,0.483915,0.345433,0.135437,0.142689,0.377488,0.818671,0.317751,0.242419,0.0700142,0.220779,0.40068,0.345597,0.200162,0.138924,0.36674,0.822287,0.320265,0.248748,0.0726098,0.219302,0.38893,0.338338,0.199628
Estrato 4_Suscriptores,0.0900301,-0.00819234,0.111767,0.22595,0.386719,1,0.466738,0.328775,0.240379,0.268652,0.180048,0.168731,0.114507,0.2304,0.389027,0.70839,0.378391,0.237802,0.233815,0.290304,0.206,0.164692,0.111268,0.221787,0.388673,0.715561,0.387071,0.241217,0.229501,0.280428,0.197826,0.168071
Estrato 5_Suscriptores,0.110469,0.00715672,0.124708,0.173218,0.293841,0.466738,1,0.477592,0.195299,0.210762,0.11842,0.182202,0.113377,0.174185,0.298466,0.417025,0.675638,0.42739,0.236236,0.265831,0.150916,0.133791,0.10925,0.168073,0.295267,0.41351,0.678072,0.427407,0.231828,0.252649,0.142645,0.139578
Estrato 6_Suscriptores,0.0871937,-0.0357081,0.0779485,-0.0254378,0.0871157,0.328775,0.477592,1,0.166302,0.0751852,-0.129618,0.0971046,0.0958467,0.0151728,0.12862,0.347592,0.430498,0.603261,0.234091,0.184669,0.0410029,0.162575,0.085919,0.00330559,0.116494,0.340471,0.430659,0.605837,0.229766,0.172763,0.0262103,0.157837
Industrial_Suscriptores,-0.0469493,0.0110199,0.087188,0.316981,0.283277,0.240379,0.195299,0.166302,1,0.361048,0.256541,0.132507,0.0858528,0.309099,0.286873,0.232679,0.188118,0.165598,0.206745,0.263524,0.254735,0.122594,0.0892796,0.312804,0.292395,0.239664,0.196753,0.169569,0.223009,0.266399,0.257152,0.128315
Comercial_Suscriptores,-0.0393589,0.0227505,0.2629,0.453604,0.483915,0.268652,0.210762,0.0751852,0.361048,1,0.385436,0.267406,0.253288,0.489146,0.48532,0.257864,0.193063,0.0857626,0.128058,0.39749,0.472113,0.311639,0.263781,0.509484,0.50571,0.273194,0.208625,0.0972485,0.139282,0.415482,0.481589,0.327569


# Exportar Base de datos

In [18]:
FinalM.to_csv('DataSui2.csv')

# Calculos Adicionales

In [9]:
for i in FinalM.columns[6:16]:
    ConsumoMedio=i.replace("_Suscriptores","_ConsumoMedio")
    FacturaMedia=i.replace("_Suscriptores","_FacturaMedia")
    TarifaMedia=i.replace("_Suscriptores","_TarifaMedia")
    Consumo=i.replace("_Suscriptores","_Consumo")
    ValorConsumo=i.replace("_Suscriptores","_ValorConsumo")
    FinalM[ConsumoMedio]=FinalM[Consumo]/V[i]
    FinalM[FacturaMedia]=FinalM[ValorConsumo]/V[i]
    FinalM[TarifaMedia]=FinalM[ValorConsumo]/(V[Consumo])

TypeError: unsupported operand type(s) for /: 'str' and 'float'

# Verificacion por Fila

In [22]:
len(FinalM)#.columns[6:16]
jkl=FinalM.iloc[5]
jkl

Unnamed: 0                                                                  5
Age                                                                      2010
Month                                                              Septiembre
Site                                                                   Urbano
Departamento                                               NORTE DE SANTANDER
Municipio                                                              TOLEDO
Empresa                     CENTRALES ELECTRICAS DEL NORTE DE SANTANDER S....
Estrato 1_Suscriptores                                                    112
Estrato 2_Suscriptores                                                   1301
Estrato 3_Suscriptores                                                    310
Estrato 4_Suscriptores                                                     12
Estrato 5_Suscriptores                                                    NaN
Estrato 6_Suscriptores                                          

# Analisis por municipio

In [17]:
V=FinalM#[FinalM["Municipio"]=="GUACARI"]
table = pd.pivot_table(V, index=['Age', 'Month', 'Site', 'Departamento', 'Municipio', 'Empresa'], values=['Estrato 1_Suscriptores', 'Estrato 2_Suscriptores',
       'Estrato 3_Suscriptores', 'Estrato 4_Suscriptores',
       'Estrato 5_Suscriptores', 'Estrato 6_Suscriptores',
       'Industrial_Suscriptores', 'Comercial_Suscriptores',
       'Oficial_Suscriptores', 'Otros_Suscriptores', 'Estrato 1_Consumo',
       'Estrato 2_Consumo', 'Estrato 3_Consumo', 'Estrato 4_Consumo',
       'Estrato 5_Consumo', 'Estrato 6_Consumo', 'Industrial_Consumo',
       'Comercial_Consumo', 'Oficial_Consumo', 'Otros_Consumo',
       'Estrato 1_Valor Consumo', 'Estrato 2_Valor Consumo',
       'Estrato 3_Valor Consumo', 'Estrato 4_Valor Consumo',
       'Estrato 5_Valor Consumo', 'Estrato 6_Valor Consumo',
       'Industrial_Valor Consumo', 'Comercial_Valor Consumo',
       'Oficial_Valor Consumo', 'Otros_Valor Consumo'],aggfunc=np.sum)
table2 = pd.pivot_table(V, index=['Age','Departamento', 'Municipio'], values=[ 'Comercial_Suscriptores',
       'Comercial_Consumo', 'Comercial_Valor Consumo'],aggfunc=np.sum)

In [37]:
table2

Comercial_Consumo  \
Age  Departamento    Municipio                                
2003 ANTIOQUIA       ABEJORRAL                    565599.00   
                     ABRIAQUI                      18658.00   
                     ALEJANDRIA                   141557.00   
                     AMAGA                       1395142.00   
                     AMALFI                       648824.00   
                     ANDES                       2253766.44   
                     ANGELOPOLIS                  163176.00   
                     ANGOSTURA                    169551.00   
                     ANORI                        214451.00   
                     ANZA                          84303.00   
                     APARTADO                   11896059.00   
                     ARBOLETES                   1016385.00   
                     ARGELIA                       88526.00   
                     ARMENIA                      652888.84   
                     BARBOSA                     2298561.87   
                     BELLO                      35542541.95   
                     BELMIRA                      283578.00   
                     BETANIA                      255939.00   
                     BETULIA                      470159.00   
                     BRICENO                       94216.00   
                     BURITICA                      68818.00   
                     CACERES                      943223.18   
                     CAICEDO                       89330.00   
                     CALDAS                      2859520.78   
                     CAMPAMENTO                        0.00   
                     CANASGORDAS                  327646.00   
                     CARACOLI                     233395.00   
                     CARAMANTA                    147697.00   
                     CAREPA                      1890059.00   
                     CAROLINA                     126723.50   
...                                                     ...   
2019 VALLE DEL CAUCA DAGUA                       1811394.00   
                     EL AGUILA                    153732.00   
                     EL CAIRO                     146932.00   
                     EL CERRITO                  2341489.00   
                     EL DOVIO                     247513.00   
                     FLORIDA                     2129001.00   
                     GINEBRA                     1035116.00   
                     GUACARI                      954857.00   
                     GUADALAJARA DE BUGA         2372972.00   
                     JAMUNDI                    11292257.00   
                     LA CUMBRE                    291019.00   
                     LA UNION                    2163842.00   
                     LA VICTORIA                  527597.00   
                     OBANDO                       376884.00   
                     PALMIRA                    47698614.00   
                     PRADERA                     1506850.00   
                     RESTREPO                     561901.00   
                     RIOFRIO                      496736.00   
                     ROLDANILLO                  2046325.00   
                     SAN PEDRO                    937820.00   
                     SEVILLA                     1882018.00   
                     TORO                         447725.00   
                     TRUJILLO                     315195.00   
                     TULUA                       4194003.00   
                     ULLOA                        173818.00   
                     VERSALLES                    192599.00   
                     VIJES                        593764.00   
                     YOTOCO                       993972.00   
                     YUMBO                       8478178.00   
                     ZARZAL                      2582886.00   

                                          Comercial_Suscriptores  \
Age 

In [36]:
table2["TarifaMedia"]=table2['Comercial_Valor Consumo']/(table2['Comercial_Consumo'])

In [38]:
table2.to_csv('Tarifas.csv')